In [140]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eric_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [141]:
import re
import numpy as np 
import pandas as pd

from pprint import pprint

In [142]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [143]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [144]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [145]:
#nltk stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.append("flutter")
stop_words.extend(['from','subject','re','edu','use'])

In [146]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:\\Users\\eric_\\Downloads\\My Project-499-f28c2476b9b8.json"

In [147]:
from google.cloud import bigquery
client = bigquery.Client()

In [148]:
sql = '''
        SELECT title, score
        FROM `bigquery-public-data.stackoverflow.tags` FULL OUTER Join `bigquery-public-data.stackoverflow.posts_questions`
        ON tag_name = tags
        where tags like'%flutter%' and score > 5
        order by score desc
'''
query_job = client.query(
    sql,
    location="US",
)
flutter = []
for row in query_job:
    flutter += row
#print(flutter)


In [149]:
#tokenize words 
def tokenize_words(txt):
    for x in txt: 
        yield(gensim.utils.simple_preprocess(str(x), deacc=True)) #deacc removes punctuations

data_words = list(tokenize_words(flutter))
#print(data_words)

In [150]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

In [151]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [152]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized)

In [153]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])

In [154]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('emulator', 1)]]

In [167]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics= 15, 
                                           random_state=20,
                                           update_every=1,
                                           chunksize=500,
                                           passes=15,
                                           alpha=15,
                                           per_word_topics=True)

In [168]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.077*"screen" + 0.062*"add" + 0.058*"image" + 0.057*"call" + 0.054*"file" '
  '+ 0.050*"item" + 0.045*"size" + 0.034*"container" + 0.034*"app" + '
  '0.026*"user"'),
 (1,
  '0.064*"image" + 0.062*"widget" + 0.062*"string" + 0.049*"pass" + '
  '0.048*"bottom" + 0.035*"message" + 0.035*"create" + 0.033*"sdk" + '
  '0.031*"open" + 0.031*"child"'),
 (2,
  '0.061*"fail" + 0.053*"firestore" + 0.052*"plugin" + 0.048*"disable" + '
  '0.046*"way" + 0.045*"display" + 0.042*"datum" + 0.041*"select" + '
  '0.039*"screen" + 0.034*"load"'),
 (3,
  '0.087*"text" + 0.085*"create" + 0.072*"image" + 0.044*"width" + '
  '0.041*"remove" + 0.036*"use" + 0.036*"padding" + 0.032*"animation" + '
  '0.028*"color" + 0.025*"widget"'),
 (4,
  '0.088*"change" + 0.080*"list" + 0.077*"function" + 0.050*"widget" + '
  '0.041*"map" + 0.040*"make" + 0.035*"effect" + 0.031*"number" + '
  '0.029*"local" + 0.024*"open"'),
 (5,
  '0.097*"error" + 0.059*"parent" + 0.053*"android_studio" + 0.041*"name" + '
  '0.035*

In [169]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.201713660166412

Coherence Score:  0.7003693014886471


In [170]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\eric_\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.067001  0.082527       1        1  6.675754
12    -0.036497  0.080985       2        1  6.675081
5      0.066399  0.212909       3        1  6.671410
1     -0.057019 -0.072849       4        1  6.669472
4     -0.078331  0.016190       5        1  6.669056
14     0.258925 -0.064469       6        1  6.668463
0     -0.015477 -0.081452       7        1  6.667961
11    -0.029438 -0.048521       8        1  6.664990
8     -0.106405  0.138125       9        1  6.664356
9      0.052271 -0.101441      10        1  6.664239
3     -0.039971 -0.161558      11        1  6.664229
10    -0.086105 -0.093426      12        1  6.662045
7      0.018702  0.059610      13        1  6.661983
2      0.202783  0.044161      14        1  6.661765
6     -0.082837 -0.010790      15        1  6.659196, topic_info=    Category       Freq    Term       Total  loglift  logprob
152  Default  51.000000   error   51.000000  30.0000  30.0000
25   Default  42.000000     add   42.000000  29.0000  29.0000
4    Default  47.000000  create   47.000000  28.0000  28.0000
7    Default  38.000000  change   38.000000  27.0000  27.0000
75   Default  69.000000   image   69.000000  26.0000  26.0000
..       ...        ...     ...         ...      ...      ...
109  Topic15   2.197170    list   22.981182   0.3617  -4.7105
67   Topic15   2.902869    make   36.241249   0.1847  -4.4320
35   Topic15   8.709501  widget  114.965210   0.1290  -3.3333
23   Topic15   2.500746  screen   36.910843   0.0173  -4.5811
75   Topic15   1.743010   image   69.103279  -0.9708  -4.9421

[541 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
204       2  0.881300      able
93       11  0.740038    access
93       15  0.246679    access
262       4  0.465401    accord
309      10  0.817917    action
...     ...       ...       ...
292      14  0.458113    window
139       9  0.310356      work
139      12  0.186213      work
139      15  0.434498      work
278      13  0.888547  wrapping

[459 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 13, 6, 2, 5, 15, 1, 12, 9, 10, 4, 11, 8, 3, 7])